In [1]:
import argparse
from collections import defaultdict, namedtuple
from io import open
import math
import os
from random import shuffle, uniform
from datetime import datetime
from future.utils import iterkeys, iteritems

from future.builtins import range
from future.utils import iteritems

In [2]:
!pip install wandb -q
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl siz

In [3]:
from sentence_transformers import SentenceTransformer
sBertModel = SentenceTransformer('all-mpnet-base-v2')

In [4]:
import wandb
wandb.login(key="4a6e96eb645ce23f4ada4b7f5106dcbaed287c63")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
def load_data(filename):
    """
    This method loads and returns the data in filename. If the data is labelled training data, it returns labels too.

    Parameters:
        filename: the location of the training or test data you want to load.

    Returns:
        data: a list of InstanceData objects from that data type and track.
        labels (optional): if you specified training data, a dict of instance_id:label pairs.
    """

    # 'data' stores a list of 'InstanceData's as values.
    data = []

    # If this is training data, then 'labels' is a dict that contains instance_ids as keys and labels as values.
    training = False
    if filename.find('train') != -1:
        training = True

    if training:
        labels = dict()

    num_exercises = 0
    print('Loading instances...')
    instance_properties = dict()

    with open(filename, 'rt') as f:
        for line in f:
            line = line.strip()

            # If there's nothing in the line, then we're done with the exercise. Print if needed, otherwise continue
            if len(line) == 0:
                num_exercises += 1
                if num_exercises % 100000 == 0:
                    print('Loaded ' + str(len(data)) + ' instances across ' + str(num_exercises) + ' exercises...')
                instance_properties = dict()

            # If the line starts with #, then we're beginning a new exercise
            elif line[0] == '#':
                if 'prompt' in line:
                    instance_properties['prompt'] = line.split(':')[1]
                else:
                    list_of_exercise_parameters = line[2:].split()
                    for exercise_parameter in list_of_exercise_parameters:
                        [key, value] = exercise_parameter.split(':')
                        if key == 'countries':
                            value = value.split('|')
                        elif key == 'days':
                            value = float(value)
                        elif key == 'time':
                            if value == 'null':
                                value = None
                            else:
                                assert '.' not in value
                                value = int(value)
                        instance_properties[key] = value

            # Otherwise we're parsing a new Instance for the current exercise
            else:
                line = line.split()
                if training:
                    assert len(line) == 7
                else:
                    assert len(line) == 6
                assert len(line[0]) == 12

                instance_properties['instance_id'] = line[0]

                instance_properties['token'] = line[1]
                instance_properties['part_of_speech'] = line[2]

                instance_properties['morphological_features'] = dict()
                for l in line[3].split('|'):
                    [key, value] = l.split('=')
                    if key == 'Person':
                        value = int(value)
                    instance_properties['morphological_features'][key] = value

                instance_properties['dependency_label'] = line[4]
                instance_properties['dependency_edge_head'] = int(line[5])
                if training:
                    label = float(line[6])
                    labels[instance_properties['instance_id']] = label
                data.append(InstanceData(instance_properties=instance_properties))

        print('Done loading ' + str(len(data)) + ' instances across ' + str(num_exercises) +
              ' exercises.\n')

    if training:
        return data, labels
    else:
        return data

In [6]:
class InstanceData(object):
    """
    A bare-bones class to store the included properties of each instance. This is meant to act as easy access to the
    data, and provides a launching point for deriving your own features from the data.
    """
    def __init__(self, instance_properties):

        # Parameters specific to this instance
        self.instance_id = instance_properties['instance_id']
        self.token = instance_properties['token']
        self.part_of_speech = instance_properties['part_of_speech']
        self.morphological_features = instance_properties['morphological_features']
        self.dependency_label = instance_properties['dependency_label']
        self.dependency_edge_head = instance_properties['dependency_edge_head']

        # Derived parameters specific to this instance
        self.exercise_index = int(self.instance_id[8:10])
        self.token_index = int(self.instance_id[10:12])

        # Derived parameters specific to this exercise
        self.exercise_id = self.instance_id[:10]

        # Parameters shared across the whole session
        self.user = instance_properties['user']
        self.countries = instance_properties['countries']
        self.days = instance_properties['days']
        self.client = instance_properties['client']
        self.session = instance_properties['session']
        self.format = instance_properties['format']
        self.time = instance_properties['time']
        self.prompt = instance_properties.get('prompt', None)

        # Derived parameters shared across the whole session
        self.session_id = self.instance_id[:8]

    def to_features(self):
        """
        Prepares those features that we wish to use in the LogisticRegression example in this file. We introduce a bias,
        and take a few included features to use. Note that this dict restructures the corresponding features of the
        input dictionary, 'instance_properties'.

        Returns:
            to_return: a representation of the features we'll use for logistic regression in a dict. A key/feature is a
                key/value pair of the original 'instance_properties' dict, and we encode this feature as 1.0 for 'hot'.
        """
        to_return = dict()

        to_return['bias'] = 1.0
        to_return['user:' + self.user] = 1.0
        to_return['format:' + self.format] = 1.0
        to_return['token:' + self.token.lower()] = 1.0

        to_return['part_of_speech:' + self.part_of_speech] = 1.0
        for morphological_feature in self.morphological_features:
            to_return['morphological_feature:' + morphological_feature] = 1.0
        to_return['dependency_label:' + self.dependency_label] = 1.0
        
        time = datetime.now()
        if(time.second %10 == 0 and time.microsecond == 0):
          print(time)
          
        return to_return

In [7]:
def load_labels(filename):
    """
    This loads labels, either the actual ones or your predictions.

    Parameters:
        filename: the filename pointing to your labels

    Returns:
        labels: a dict of instance_ids as keys and labels between 0 and 1 as values
    """
    labels = dict()

    with open(filename, 'rt') as f:
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            else:
                line = line.split()
            instance_id = line[0]
            label = float(line[1])
            labels[instance_id] = label
    return labels


def compute_acc(actual, predicted):
    """
    Computes the accuracy of your predictions, using 0.5 as a cutoff.

    Note that these inputs are lists, not dicts; they assume that actual and predicted are in the same order.

    Parameters (here and below):
        actual: a list of the actual labels
        predicted: a list of your predicted labels
    """
    num = len(actual)
    acc = 0.
    for i in range(num):
        if round(actual[i], 0) == round(predicted[i], 0):
            acc += 1.
    acc /= num
    return acc


def compute_avg_log_loss(actual, predicted):
    """
    Computes the average log loss of your predictions.
    """
    num = len(actual)
    loss = 0.

    for i in range(num):
        p = predicted[i] if actual[i] > .5 else 1. - predicted[i]
        loss -= math.log(p)
    loss /= num
    return loss


def compute_auroc(actual, predicted):
    """
    Computes the area under the receiver-operator characteristic curve.
    This code a rewriting of code by Ben Hamner, available here:
    https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/auc.py
    """
    num = len(actual)
    temp = sorted([[predicted[i], actual[i]] for i in range(num)], reverse=True)

    sorted_predicted = [row[0] for row in temp]
    sorted_actual = [row[1] for row in temp]

    sorted_posterior = sorted(zip(sorted_predicted, range(len(sorted_predicted))))
    r = [0 for k in sorted_predicted]
    cur_val = sorted_posterior[0][0]
    last_rank = 0
    for i in range(len(sorted_posterior)):
        if cur_val != sorted_posterior[i][0]:
            cur_val = sorted_posterior[i][0]
            for j in range(last_rank, i):
                r[sorted_posterior[j][1]] = float(last_rank+1+i)/2.0
            last_rank = i
        if i==len(sorted_posterior)-1:
            for j in range(last_rank, i+1):
                r[sorted_posterior[j][1]] = float(last_rank+i+2)/2.0

    num_positive = len([0 for x in sorted_actual if x == 1])
    num_negative = num - num_positive
    sum_positive = sum([r[i] for i in range(len(r)) if sorted_actual[i] == 1])
    auroc = ((sum_positive - num_positive * (num_positive + 1) / 2.0) / (num_negative * num_positive))

    return auroc


def compute_f1(actual, predicted, cutoff = 0.5):
    """
    Computes the F1 score of your predictions. Note that we use 0.5 as the cutoff here.
    """
    num = len(actual)

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0

    for i in range(num):
        if actual[i] >= cutoff and predicted[i] >= cutoff:
            true_positives += 1
        elif actual[i] < cutoff and predicted[i] >= cutoff:
            false_positives += 1
        elif actual[i] >= cutoff and predicted[i] < cutoff:
            false_negatives += 1
        else:
            true_negatives += 1

    try:
        precision = true_positives / (true_positives + false_positives)
        print(precision)
        recall = true_positives / (true_positives + false_negatives)
        print(recall)
        F1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        F1 = 0.0

    return F1


def evaluate_metrics(actual, predicted):
    """
    This computes and returns a dictionary of notable evaluation metrics for your predicted labels.
    """
    acc = compute_acc(actual, predicted)
    avg_log_loss = compute_avg_log_loss(actual, predicted)
    auroc = compute_auroc(actual, predicted)
    F1 = compute_f1(actual, predicted)

    return  acc, avg_log_loss,  auroc, F1


def test_metrics():
    actual = [1, 0, 0, 1, 1, 0, 0, 1, 0, 1]
    predicted = [0.8, 0.2, 0.6, 0.3, 0.1, 0.2, 0.3, 0.9, 0.2, 0.7]
    metrics = evaluate_metrics(actual, predicted)
    metrics = {key: round(metrics[key], 3) for key in iterkeys(metrics)}
    assert metrics['accuracy'] == 0.700
    assert metrics['avglogloss'] == 0.613
    assert metrics['auroc'] == 0.740
    assert metrics['F1'] == 0.667
    print('Verified that our environment is calculating metrics correctly.')

In [9]:
training_data, training_labels = load_data("/content/en_es/en_es.slam.20190204.train")

Loading instances...
Loaded 317049 instances across 100000 exercises...
Loaded 635368 instances across 200000 exercises...
Loaded 951536 instances across 300000 exercises...
Loaded 1271940 instances across 400000 exercises...
Loaded 1591344 instances across 500000 exercises...
Loaded 1911212 instances across 600000 exercises...
Loaded 2227444 instances across 700000 exercises...
Loaded 2546704 instances across 800000 exercises...
Done loading 2622957 instances across 824012 exercises.



In [8]:
test_data = load_data("/content/en_es/en_es.slam.20190204.dev")

Loading instances...


KeyboardInterrupt: ignored

In [17]:
def convert_data_for_processing(dataset, isTrain, word_dict, pos_dict, format_dict, dependency_label_dict, morphological_feature_dict):
#Convert into data that can be used to train the BKT agent
  if isTrain:
    exercices = {}
    instanceIdExerciseMap = {}
    word_dict = {
        'unk': 0
    }
    pos_dict = {
        'unk': 0
    }
    format_dict = {
        'unk': 0
    }
    dependency_label_dict = {
        'unk': 0
    }
    morphological_feature_dict = {
        'unk': 0
    }

    unique_word_index = 1;
    unique_pos_index = 1;
    unique_format_index = 1;
    unique_dependency_label_index = 1;
    unique_morphological_feature_index = 1;

  for instance in dataset:
    user = instance.user
    instance_id = instance.instance_id[:-2]
    if user not in exercices:
      exercices[user] = {}
    if instance_id not in exercices[user] :
      exercices[user][instance_id] = []
    if instance_id not in instanceIdExerciseMap:
      instanceIdExerciseMap[instance_id] = []
    
    token = instance.token.lower()
    part_of_speech =  instance.part_of_speech.lower()
    format = instance.format
    dependency_label = instance.dependency_label
    morphological_features = instance.morphological_features

    token_info = []

    if token not in word_dict:
      if isTrain:
        word_dict[token] = unique_word_index
        unique_word_index += 1
      else:
        token = '<unk>'
    token_info.append(word_dict[token])

    instanceIdExerciseMap[instance_id].append(token)
    if part_of_speech not in pos_dict:
      if isTrain:
        pos_dict[part_of_speech] = unique_pos_index
        unique_pos_index += 1
      else:
        part_of_speech = 'unk'
    token_info.append(pos_dict[part_of_speech])

    if format not in format_dict:
      if isTrain:
        format_dict[format] = unique_format_index
        unique_format_index += 1
      else:
        format = 'unk'
    token_info.append(format_dict[format])

    if dependency_label not in dependency_label_dict:
      if isTrain:
        dependency_label_dict[dependency_label] = unique_dependency_label_index
        unique_dependency_label_index += 1
      else:
        dependency_label = 'unk'
    token_info.append(dependency_label_dict[dependency_label])


    morphology = []
    for feature_key, feature_val in morphological_features.items():
      key_val = str(feature_key) + ":" + str(feature_val)
      if key_val in morphological_feature_dict:
        morphology.append(morphological_feature_dict[key_val])
      else:
        if isTrain:
          morphological_feature_dict[key_val]= unique_morphological_feature_index
          unique_morphological_feature_index +=1 
        else:
          key_val = 'unk'
        morphology.append(morphological_feature_dict[key_val])

    token_info.append(morphology)
    exercices[user][instance_id].append(token_info)

    #Done Processing all data. Now final processing
  exercices_merged = []
  for user in exercices:
    for instance_id in exercices[user]:
      exercices_merged.append(exercices[user][instance_id])

  for instance_id in instanceIdExerciseMap:
    instanceIdExerciseMap[instance_id] = " ".join(instanceIdExerciseMap[instance_id])
  
  encodings = sBertModel.encode(list(instanceIdExerciseMap.values()), show_progress_bar = True)
  instanceIdExerciseEmbeddingMap = {}
  for i, instance_id in enumerate(list(instanceIdExerciseMap.keys())):
    instanceIdExerciseEmbeddingMap[instance_id] = encodings[i]

  return exercices_merged, instanceIdExerciseMap,instanceIdExerciseEmbeddingMap, word_dict, pos_dict, format_dict, dependency_label_dict, morphological_feature_dict

In [194]:
len(training_data)

2622957

In [ ]:
exercices_merged, instanceIdExerciseMap,instanceIdExerciseEmbeddingMap, word_dict, pos_dict, format_dict, dependency_label_dict, morphological_feature_dict = convert_data_for_processing(training_data[0:2620000], True, None, None, None, None, None)

Batches:   0%|          | 0/25723 [00:00<?, ?it/s]

In [22]:
import numpy as np

## Test BKT 

In [25]:
import numpy as np

In [133]:
class BKTLearner(object):
    
    def __init__(self, token_state_size, pos_state_size, format_state_size, dependency_state_size, morphological_state_size, slip_prob, transition_prob, guess_prob):
        self.token_state_size = token_state_size
        self.pos_state_size = pos_state_size
        self.format_state_size = format_state_size
        self.dependency_state_size = dependency_state_size
        self.morphological_state_size = morphological_state_size

        self.token_state = np.full(self.token_state_size,0.2, dtype = np.float32)
        self.pos_state = np.full(self.pos_state_size,0.2,dtype = np.float32)
        self.format_state = np.full(self.format_state_size,0.2,dtype = np.float32)
        self.dependency_state = np.full(self.dependency_state_size,0.2,dtype = np.float32)
        self.morphological_state = np.full(self.morphological_state_size,0.2,dtype = np.float32)


        self.slip_prob = slip_prob
        self.transition_prob = transition_prob
        self.guess_prob = guess_prob
    
    def reset(self):
        self.token_state = np.full(self.token_state_size,0.2, dtype = np.float32)
        self.pos_state = np.full(self.pos_state_size,0.2,dtype = np.float32)
        self.format_state = np.full(self.format_state_size,0.2,dtype = np.float32)
        self.dependency_state = np.full(self.dependency_state_size,0.2,dtype = np.float32)
        self.morphological_state = np.full(self.morphological_state_size,0.2,dtype = np.float32)
        

    def getNetLearnedProb(self,token_info):
      token_index = token_info[0]
      pos_index = token_info[1]
      format_index = token_info[2]
      dep_index = token_info[3]
      morphological_indices = token_info[4]
      net_learned = self.token_state[token_index] * self.pos_state[pos_index] #* self.format_state[format_index] * self.dependency_state[dep_index]
      for index in morphological_indices:
       net_learned *= self.morphological_state[index]

      return net_learned

    def predictAnswerProbabilities(self, input):
        answer = []
        for token_info in input:
            net_learned = self.getNetLearnedProb(token_info)
            p_correct = net_learned * (1 - self.slip_prob) + (1 - net_learned) * self.guess_prob 
            answer.append(p_correct)
        return np.array(answer)

    def predictAnswer(self, input):
        answer = []
        for token_info in input:
            net_learned = self.getNetLearnedProb(token_info)
            p_correct = net_learned * (1 - self.slip_prob) + (1 - net_learned) * self.guess_prob 
            value = np.random.choice(np.array([0,1]), p = np.array([1 - p_correct, p_correct]))
            answer.append(value)
        return np.array(answer)
    
    def getPosterior(self, prob, output_correctness):
      if output_correctness == 1:
        posterior = prob*(1 - self.slip_prob) / (prob*(1 - self.slip_prob) + (1 - prob)*self.guess_prob)
      else:
        posterior = prob*(self.slip_prob) / (prob*(self.slip_prob) + (1 - prob)*(1 - self.guess_prob))
      return posterior

    def updateKnowledgeState(self, output_correctness, input):
        i = 0
        for token_info in input:
          #print("Updating token state for : " + str(token_info[0]))
          token_posterior = self.getPosterior(self.token_state[token_info[0]], output_correctness[i])
          self.token_state[token_info[0]] = token_posterior + (1 - token_posterior) * self.transition_prob          

          pos_posterior = self.getPosterior(self.pos_state[token_info[1]], output_correctness[i])
          self.pos_state[token_info[1]] = pos_posterior + (1 - pos_posterior) * self.transition_prob

          format_posterior = self.getPosterior(self.format_state[token_info[2]], output_correctness[i])
          self.format_state[token_info[2]] = format_posterior + (1 - format_posterior) *  self.transition_prob

          dep_posterior = self.getPosterior(self.dependency_state[token_info[3]], output_correctness[i])
          self.dependency_state[token_info[3]] = dep_posterior + (1 - dep_posterior) * self.transition_prob

          for index in token_info[4]:
           morpho_posterior = self.getPosterior(self.morphological_state[index], output_correctness[i])
           self.morphological_state[index] = morpho_posterior + (1 - morpho_posterior) * self.transition_prob
          i += 1 

    def trainOneSet(self, excercises):
        for exercise in excercises:
            answer_correctness = self.predictAnswer(exercise)
            self.updateKnowledgeState(answer_correctness, exercise)
    
    def trainOneExercise(self, exercise):
      answer_correctness = self.predictAnswer(exercise)
      self.updateKnowledgeState(answer_correctness, exercise)

    def testOneSetProbabilities(self, excercises):
        answer_correctness = []
        for exercise in excercises:
            answer_correctness_ex = self.predictAnswerProbabilities(exercise)
            answer_correctness.append(answer_correctness_ex)
        return np.array(answer_correctness)

    def testOneSet(self, excercises):
        answer_correctness = []
        for exercise in excercises:
            answer_correctness_ex = self.predictAnswer(exercise)
            answer_correctness.append(answer_correctness_ex)
        return np.array(answer_correctness)
        
    def computeAccuracyForTest(self, test_response):
        correct = 0;
        total = 0;
        for exercise in test_response:
            for token in exercise:
                correct += token
                total += 1
        
        if(total == 0):
          return 0

        return float(correct)/total * 100
    
    def train(self, exercices_all, train_duration, test_duration):
        i = 0;
        accuracy = 0
        batch = 0
        cummulative_reward = 0;
        
        while i < len(exercices_all):
            if(train_duration + i < len(exercices_all)):
              train_batch = exercices_all[i:train_duration + i]
            else: 
              train_batch = exercices_all[i:]
            i += train_duration
            if(i + test_duration < len(exercices_all)):
              test_batch = exercices_all[i:i + test_duration]
            else:
               test_batch = exercices_all[i:]
            i += test_duration

            answer_correctness_before = self.testOneSetProbabilities(test_batch)
            #print(answer_correctness_before)
            answer_correctness_before = [np.where(answer_correctness_before[i] >= 0.5 , 1, 0 ) for i in range(len(answer_correctness_before))]
            self.trainOneSet(train_batch)
            answer_correctness = self.testOneSetProbabilities(test_batch)
            answer_correctness = [np.where(answer_correctness[i] >= 0.5 , 1, 0 ) for i in range(len(answer_correctness))]

            accuracy_before = self.computeAccuracyForTest(answer_correctness_before) 
            accuracy = self.computeAccuracyForTest(answer_correctness) 
            print("Batch + " + str(batch) + " " + " correct before: " + str(accuracy_before) + " correct after: " + str(accuracy))
            cummulative_reward += accuracy - accuracy_before
            wandb.log({'Batch Accuracy After': accuracy, 'Batch Accuracy Before': accuracy_before, "reward": accuracy - accuracy_before, "cummulative reward":cummulative_reward})
            batch += 1

# Simple RL Learner

In [180]:
class RLEnvironment(object):

  def __init__(self, bkt_learner):
    self.bkt_learner = bkt_learner
  
  def start_episode(self, test_set):
    self.test_set = test_set
    test_correctness = self.bkt_learner.testOneSetProbabilities(test_set)
    test_correctness = [np.where(test_correctness[i] >= 0.5 , 1, 0 ) for i in range(len(test_correctness))]
    self.start_score = self.bkt_learner.computeAccuracyForTest(test_correctness) #ToDo: Calculate start score on test set here.
 
  
  def reset(self):
    self.bkt_learner.reset()

  def get_state(self):
    return np.concatenate((self.bkt_learner.token_state, 
                          self.bkt_learner.pos_state, self.bkt_learner.format_state ,
                          self.bkt_learner.dependency_state,
                          self.bkt_learner.morphological_state), axis = 0)
  
  def get_available_actions(self):
    return self.train_set_embeddings, self.train_set

  def step(self, action, isEpisodeEnd):
    if not isEpisodeEnd:
      self.bkt_learner.trainOneExercise(action)
    
    if not isEpisodeEnd:
      return self.get_state(), -1
    else:
      test_correctness = self.bkt_learner.testOneSetProbabilities(self.test_set)
      test_correctness = [np.where(test_correctness[i] >= 0.5 , 1, 0 ) for i in range(len(test_correctness))]
      score = self.bkt_learner.computeAccuracyForTest(test_correctness)
      #print("Score : " + str(score))
      return self.get_state(), score - self.start_score #ToDo: Calculate end score on test set here.

  

In [184]:
class RLTestEnvironment(object):

  def __init__(self, env, exercise_list, exercise_embeddings, exercise_embedding_size, batch_size, epsilon, lr, gamma, num_episodes, max_iter):
    self.env = env
    self.exercise_embedding_size = exercise_embedding_size
    self.lr = lr
    self.gamma = gamma
    self.num_episodes = num_episodes
    self.exercise_list = exercise_list
    self.exercise_embeddings = exercise_embeddings
    self.batch_size = batch_size
    self.max_iter = max_iter

    self.weights = np.zeros((exercise_embedding_size +  len(env.get_state()) + 1) , dtype=float)
    self.num_batches = exercise_embeddings.shape[0]//batch_size
    
    self.epsilon = epsilon

  def getQValueForActions(self, actions):
    #Actions are embeddings
    current_state = self.env.get_state()
    q_values = []

    for i, action in enumerate(actions):
      #print(action)
      #print(current_state)
      #print()
      concat_state = np.append(np.array([1]), np.concatenate((current_state,action)))
      q_val = np.sum(self.weights * concat_state)
      q_values.append(q_val)
    
    end_action_concat = np.append(np.array([1]), np.concatenate((current_state,np.zeros(self.exercise_embedding_size))))
    q_val = np.sum(self.weights * end_action_concat)
    q_values.append(q_val)

    return np.array(q_values)

  def run_one_episode(self):
    net_reward = 0;
    
    for batchIdx in range(self.num_batches):
      batch = self.exercise_list[batchIdx*self.batch_size:batchIdx*self.batch_size + self.batch_size]
      batch_embeddings = self.exercise_embeddings[batchIdx*self.batch_size:batchIdx*self.batch_size + self.batch_size]
      
      train_set = batch[0:int(0.9 * batch.shape[0])]
      test_set =  batch[int(0.9 * batch.shape[0]):]

      train_set_embeddings = batch_embeddings[0:int(0.9 * batch.shape[0])]
      test_set_embeddings = batch_embeddings[int(0.9 * batch.shape[0]):]

      self.env.start_episode(test_set)

      for iter in range(self.max_iter):
        #print("here")
        choice = np.random.choice([0,1], p = [self.epsilon, 1 - self.epsilon])

        #Find the Qvalue for all actions. 
        q_value = []

        next_action = 0;
            
        if(choice == 0):
            #select action randomly
            next_action = np.random.choice(range(len(train_set_embeddings) + 1))
        else:
            #select the optimal action
            q_values = self.getQValueForActions(train_set_embeddings)
            next_action = np.argmax(q_values);
        

        episode_end = False
        if(next_action == train_set.shape[0]):
          episode_end = True
          next_state, reward = self.env.step(0, True)
          q_current_state = self.getQValueForActions(np.array([np.zeros(self.exercise_embedding_size)]))[0]
          current_state_with_bias = np.append(np.array([1]), np.concatenate((self.env.get_state(),np.zeros(self.exercise_embedding_size))))
        else:
          next_state, reward = self.env.step(train_set[next_action], False)
          q_current_state = self.getQValueForActions(np.array([train_set_embeddings[next_action]]))[0]
          current_state_with_bias = np.append(np.array([1]), np.concatenate((self.env.get_state(),train_set_embeddings[next_action])))
        if(reward != -1):
          net_reward += reward

        #train_set = np.delete(train_set, next_action, axis = 0)
        #train_set_embeddings = np.delete(train_set_embeddings, next_action, axis = 0)

        # if(train_set_embeddings.shape[0] == 0):
        #   next_state, reward = self.env.step(0, True)
        #   net_reward += reward
        #   break;

        next_q_values = self.getQValueForActions(train_set_embeddings)
        max_q_next_state = np.max(next_q_values)

        #Update weights
        self.weights = self.weights - (self.lr * ((q_current_state - (reward + self.gamma * max_q_next_state)) * current_state_with_bias))

        if(episode_end):
          break

    return net_reward
  def run_training(self):
    reward_over_episodes = []
    for episode in range(self.num_episodes):
      self.env.reset()
      reward = self.run_one_episode()
      reward_over_episodes.append(reward)
      print(reward)
      #print(self.env.bkt_learner.token_state)
    
    return reward_over_episodes

# RL Testing

In [191]:
#np.random.shuffle(all_ex)
learner = BKTLearner(len(word_dict),len(pos_dict), len(format_dict), len(dependency_label_dict),len(morphological_feature_dict), 0.1, 0.1, 0.001)
rlEnv = RLEnvironment(learner)
rlTestEnv = RLTestEnvironment(env = rlEnv, exercise_list = np.array(exercices_merged), exercise_embeddings = np.array(list(instanceIdExerciseEmbeddingMap.values())), exercise_embedding_size = 768, batch_size = 50, epsilon=0.1, lr=0.1, gamma=0.1, num_episodes = 10, max_iter = 100)
#learner.train(all_ex, 50, 20)

<ipython-input-191-5c6b2d70ad9d>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  rlTestEnv = RLTestEnvironment(env = rlEnv, exercise_list = np.array(exercices_merged), exercise_embeddings = np.array(list(instanceIdExerciseEmbeddingMap.values())), exercise_embedding_size = 768, batch_size = 50, epsilon=0.1, lr=0.1, gamma=0.1, num_episodes = 10, max_iter = 100)


In [192]:
rlTestEnv.run_training()

<ipython-input-133-6dc27b004243>:100: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(answer_correctness)
/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-184-4f0b07b97de2>:96: RuntimeWarning: invalid value encountered in double_scalars
  self.weights = self.weights - (self.lr * ((q_current_state - (reward + self.gamma * max_q_next_state)) * current_state_with_bias))


14.285714285714285
15.789473684210524
4.761904761904759
16.666666666666664
0.0
13.333333333333343
0.0
6.25
0.0
0.0


[14.285714285714285,
 15.789473684210524,
 4.761904761904759,
 16.666666666666664,
 0.0,
 13.333333333333343,
 0.0,
 6.25,
 0.0,
 0.0]

# **Later**

In [ ]:
labels = load_labels("/content/en_es/en_es.slam.20190204.dev.key")
actual = []

for instance_id in iterkeys(labels):
    try:
        actual.append(labels[instance_id])
    except KeyError:
        print('No prediction for instance ID ' + instance_id + '!')
final_result = []
for user in test_exercices_merged:
  final_result.append(results[user])

auroc = compute_auroc(actual,np.concatenate(final_result))
f1 = compute_f1(actual,np.concatenate(final_result), 0.5)
acc = compute_acc(actual,np.concatenate(final_result))
print("Acc " + str(acc) + "auroc " + str(auroc) + " f1 : " + str(f1))